In [1]:
%cd ~/github/agn_lf/
import source.astro_functions as af
import source.table_functions as tf
import source.lf_xi2 as xi2
import source.lf_vmax as vmax
import numpy as np
import matplotlib.pyplot as plt

/Users/runburg/github/agn_lf


In [2]:
# use table with agn_flag
from astropy.io import fits
import astropy.table as table
catalog = fits.open("data/Anna_Nick_Tractor_photometry_samplev9_MASTER.fits", memmap=True)
# ct = table.Table(catalog[1].data)
# flagged_agn = ct[np.where(ct['agn_flag'] == 1)]
# flagged_agn = table.Table(flagged_agn, masked=True, copy=False)
# # for col in flagged_agn.columns.values():
# #     try:
# #         col.mask = np.isnan(col)
# #     except TypeError:
# #         continue
# flagged_agn.write('./data/flagged_agn.fits', format='fits')
flagged_agn = table.Table.read('./data/flagged_agn.fits')